# Project 02 - Due Friday, November 11 at 12pm

*Objectives*: Gain experience loading a transactional dataset into a relational database model you define yourself and using SQL to explore its contents. Transform the data into a star schema, documenting the schema visually, and explore the transformed data analytically by writing and executing a number of SQL queries using common syntax and functions and describing your findings.  Gain practice working on these tasks with a partner.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries, diagrams, and charts in the space provided, unless a text answer is requested.  The notebook itself should be completely reproducible at datanotebook.org, from start to finish: another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Attestation*: **Work in pairs**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Friday, November 11, 12pm.  One member of each pair must submit your notebook to Blackboard; you should not both submit it separately.  You may each push a copy to your own GitHub repository.


## Setup - select and obtain data

The US City Open Data Census has a variety of [transactional data from all over the country](http://us-city.census.okfn.org/).  Select one topic and one city from this Census, download the data, and explain your reason for this choice before proceeding.  Provide a link to the web page for the data set you've chosen.

I strongly encourage you to select a dataset from among the following topics:  Crime, Property Assessment, Campaign Finance Contributions, Service Requests (311), and Web Analytics.

By *transactional* data I mean records at the grain of one event per business process.  In the case of Service Requests (311), for example, that would require data at the level of each individual report of a service request.  If this isn't clear, think about the bike trip data - each individual ride was included - and look for data at that level of event/process specificity.  Avoid data like the Connecticut boating data, which was summarized by year, not individual transactions.

Please aim for a dataset of at least 10,000 individual records, but less than 250,000 records.  A little more or a little less is okay, but try to stay within these limits.

## Problem 1 - examine the data (20 points)

Use `csvstat`, `pandas`, or other tools as you see fit and observe the contents of your dataset.  Review important points from any metadata provided.  Describe what you see:  What do the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some opportunities, challenges, or questions you would like to ask?

Note: Joining the facts table to dimension tables query will take a longer time to run but it works.

### Our Approach to Problem 1:

In the next few blocks of code, we established a connection to the database, read in a CSV file, examined the number of records, examined the names of the variables, examined sample stats for a sample of the data, and began to explore the data with a few command line commands.

In [1]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [2]:
!createdb -U dbuser BPD_arrests

In [3]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/Divya-Gorwara/master/BPD_Arrests.csv

--2016-11-11 08:29:32--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/Divya-Gorwara/master/BPD_Arrests.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19703253 (19M) [text/plain]
Saving to: ‘BPD_Arrests.csv’

BPD_Arrests.csv     100%[=====================>]  18.79M  71.3MB/s   in 0.3s   

2016-11-11 08:29:33 (71.3 MB/s) - ‘BPD_Arrests.csv’ saved [19703253/19703253]



In [4]:
!wc -l BPD_Arrests.csv

129026 BPD_Arrests.csv


In [5]:
!csvcut -n BPD_Arrests.csv

  1: Arrest
  2: Age
  3: Sex
  4: Race
  5: ArrestDate
  6: ArrestTime
  7: ArrestLocation
  8: IncidentOffense
  9: IncidentLocation
 10: Charge
 11: ChargeDescription
 12: District
 13: Post
 14: Neighborhood
 15: Location 1


In [6]:
!shuf -n 100 BPD_Arrests.csv | csvstat

  1. 16016408
	<class 'int'>
	Nulls: True
	Min: 13610635
	Max: 16128732
	Sum: 1414400471
	Mean: 14581448.154639175
	Median: 14106870
	Standard Deviation: 917725.6258565299
	Unique values: 97
  2. 24
	<class 'int'>
	Nulls: False
	Min: 18
	Max: 65
	Sum: 3302
	Mean: 33.35353535353536
	Median: 29
	Standard Deviation: 12.83407170745841
	Unique values: 37
	5 most frequent values:
		25:	8
		28:	6
		22:	6
		21:	5
		20:	5
  3. M
	<class 'str'>
	Nulls: False
	Values: M, F
  4. W
	<class 'str'>
	Nulls: False
	Values: U, A, W, B
  5. 02/03/2016
	<class 'datetime.date'>
	Nulls: False
	Min: 2013-01-01
	Max: 2016-09-07
	Unique values: 92
	5 most frequent values:
		2013-10-18:	2
		2016-08-31:	2
		2015-08-14:	2
		2014-01-28:	2
		2014-04-14:	2
  6. 13:00
	<class 'str'>
	Nulls: False
	Unique values: 83
	5 most frequent values:
		17:00:	3
		20:00:	3
		00:00:	2
		22:00:	2
		19:30:	2
	Max length: 5
  7. 4000 8TH ST
	<class 'str'>
	Nulls: True
	Unique values: 57
	5 most frequent values:
		1600 W NORTH AVE:	2

In [7]:
!csvcut -c3,6,9 BPD_Arrests.csv | head -10 | csvlook

|------+------------+----------------------------|
|  Sex | ArrestTime | IncidentLocation           |
|------+------------+----------------------------|
|  M   | 23:00      | 400 N FRANKLINTOWN RD      |
|  M   | 23:00      | W NORTH AV & N CHARLES ST  |
|  M   | 22:45      | 2300 JEFFERSON ST          |
|  M   | 22:40      | 3500 NOBLE ST              |
|  F   | 22:22      | 2100 WILHELM ST            |
|  M   | 22:00      | 0 N HOWARD ST              |
|  M   | 21:50      | 500 E PATAPSCO AVE         |
|  M   | 21:50      | 1500 HARFORD AVE           |
|  M   | 21:15      | 2100 HARFORD RD            |
|------+------------+----------------------------|


In [8]:
!csvcut -c3,6,9 BPD_Arrests.csv | csvgrep -c3 -m '2100 HARFORD RD' | csvsort -c2 | head -10 | csvlook

|------+------------+-------------------|
|  Sex | ArrestTime | IncidentLocation  |
|------+------------+-------------------|
|  M   | 13.30      | 2100 HARFORD RD   |
|  M   | 15.40      | 2100 HARFORD RD   |
|  M   | 21:15      | 2100 HARFORD RD   |
|  M   | 22.00      | 2100 HARFORD RD   |
|------+------------+-------------------|


## Problem 2 - define a database model, load the data, and explore (20 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset as it is in its raw form into the database, and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Explore the data to zero in on a few themes you would like to further study analytically.  Discuss columns that present opportunities for extraction into dimensions, and identify the specific columns that contain facts you want to measure.

### Our Approach to Problem 2:

Using the sample stats from problem 1, we estimated how much space to allocate to each of the variables that we wanted to include in our database from the orgininal data file.  Once we created the bpd_table, we made a copy of it so that we could work from a version that was not the original (in case we ever had to reference the original file).

We then ran a query to verify that the number of rows in the table matched the number of rows in the original data set (129025).  We then ran a few queries to further explore the data, such as identifying how many males/females have been arrested, what types of incidents have occured, and what the locations of the incidents have been.

In [4]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [5]:
%sql postgresql://dbuser@localhost:5432/BPD_arrests

'Connected: dbuser@BPD_arrests'

In [6]:
%%sql
DROP TABLE IF EXISTS bpd_arrests;
CREATE TABLE bpd_arrests(
  ARREST INTEGER,
  AGE INTEGER,
  SEX CHAR(6),
  RACE VARCHAR(64),
  ARREST_DATE TIMESTAMP,
  ARREST_TIME varchar(64),
  ARREST_LOCATION VARCHAR(64),
  INCIDENT_OFFENSE VARCHAR(64),
  INCIDENT_LOCATION VARCHAR(64),
  CHARGE VARCHAR(64),
  CHARGE_DESC VARCHAR(64),
  DISTRICT VARCHAR(64),
  POST INTEGER,
  NEIGHBORHOOD VARCHAR(64),
  LOCATION_COORDINATES VARCHAR(64)
)

Done.
Done.


[]

In [7]:
!pwd

/home/jovyan/work


In [8]:
%%sql
COPY bpd_arrests FROM '/home/jovyan/work/BPD_Arrests.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

129025 rows affected.


[]

In [9]:
%%sql
SELECT COUNT(*) FROM bpd_arrests;

1 rows affected.


count
129025


In [15]:
%%sql
SELECT SEX, COUNT(*) AS SEX_COUNT
FROM bpd_arrests
GROUP BY SEX;

2 rows affected.


sex,sex_count
F,24547
M,104478


In [16]:
%%sql
SELECT INCIDENT_OFFENSE, COUNT(*) AS INCIDENT_OFFENSE_COUNT
FROM bpd_arrests
GROUP BY INCIDENT_OFFENSE
ORDER BY INCIDENT_OFFENSE_COUNT DESC
LIMIT 10;

10 rows affected.


incident_offense,incident_offense_count
Unknown Offense,80719
87-Narcotics,12294
4E-Common Assault,6304
UNKNOWN OFFENSE,3209
87O-Narcotics (Outside),2602
6C-Larceny- Shoplifting,2222
79-Other,1959
24-Towed Vehicle,1581
97-Search & Seizure,1544
4C-Agg. Asslt.- Oth.,1466


In [17]:
%%sql
SELECT ARREST_LOCATION, COUNT(*)
FROM bpd_arrests
WHERE ARREST_LOCATION LIKE '%HIGHLAND%'
GROUP BY ARREST_LOCATION
ORDER BY ARREST_LOCATION;

16 rows affected.


arrest_location,count
100 N HIGHLAND AVE,23
100 S HIGHLAND AVE,18
100 S HIGHLAND ST,2
1400 S HIGHLAND AVE,2
200 N HIGHLAND AVE,50
200 S HIGHLAND AVE,5
300 N HIGHLAND AVE,1
300 S HIGHLAND AVE,2
400 N HIGHLAND AVE,25
400 S HIGHLAND AVE,4


## Problem 3 - define a star schema, and load your data into it (30 points)

Define a star schema consisting of at least one fact table and at least two dimensions.  Transform your source data into the new fact table and dimensions.  Discuss your key modeling decisions.

Document your star schema with a diagram.  There several tools available for this, such as Google Docs and Microsoft Visio.

### Our Approach to Problem 3:

We first finished making our copy of the bpd_arrests table and continued to use that version throughout the rest of the assignment.  We then went through the process of updating missing and null values for the variables with which we identified these issues.

Next, we created three dimension tables (person, time, location), each with a primary key that would eventually be used to link the dimension table to the fact table.  In addition to the primary key, the dimension tables also contained the variables associated with the particular dimension.  In some circumstances, it would have been great to create a more granular grain, but for this assignment it was stated that we could keep the variables relatively high level.

After we created the dimension tables, we created the fact table (arrests_facts).  The only fact in this table is "arrest"; the other variables are the keys that match to the various dimension tables.  We also made statements declaring that the "key" variables were the foreign keys used to link to the dimension tables.

Lastly, we filled in the dimension table with an insert statement and declared how the fact and dimension tables were linked together with the copy_bpd_arrests table.  We then wrote a simple query to verify that the arrests_facts table populated correctly.

In [10]:
%%sql
SELECT *
INTO copy_bpd_arrests 
FROM bpd_arrests;

129025 rows affected.


[]

In [11]:
%%sql
UPDATE copy_bpd_arrests
SET ARREST = 1
WHERE ARREST IS NULL;

6926 rows affected.


[]

In [12]:
%%sql
UPDATE copy_bpd_arrests
SET AGE = 1
WHERE AGE IS NULL;

28 rows affected.


[]

In [13]:
%%sql
UPDATE copy_bpd_arrests
SET AGE = 1
WHERE AGE = 0;

28 rows affected.


[]

In [14]:
%%sql
UPDATE copy_bpd_arrests
SET ARREST_LOCATION = 'NOT_REPORTED'
WHERE ARREST_LOCATION IS NULL;

51324 rows affected.


[]

In [15]:
%%sql
UPDATE copy_bpd_arrests
SET INCIDENT_LOCATION = 'NOT_REPORTED'
WHERE INCIDENT_LOCATION IS NULL;

52934 rows affected.


[]

In [16]:
%%sql
DROP TABLE IF EXISTS person_dimension;
CREATE TABLE person_dimension(
  person_key SERIAL PRIMARY KEY,
  AGE INTEGER,
  SEX CHAR(6),
  RACE VARCHAR(64)
)

Done.
Done.


[]

In [17]:
%%sql
INSERT INTO person_dimension (AGE, SEX, RACE)
SELECT DISTINCT AGE, SEX, RACE 
FROM copy_bpd_arrests;

499 rows affected.


[]

In [18]:
%%sql
DROP TABLE IF EXISTS time_dimension;
CREATE TABLE time_dimension(
  time_key SERIAL PRIMARY KEY,
  ARREST_DATE TIMESTAMP, 
  ARREST_TIME varchar(64)
)

Done.
Done.


[]

In [19]:
%%sql
INSERT INTO time_dimension (ARREST_DATE, ARREST_TIME)
SELECT DISTINCT ARREST_DATE, ARREST_TIME 
FROM copy_bpd_arrests;

89774 rows affected.


[]

In [20]:
%%sql
DROP TABLE IF EXISTS location_dimension;
CREATE TABLE location_dimension(
  location_key SERIAL PRIMARY KEY,
  ARREST_LOCATION VARCHAR(64),
  INCIDENT_LOCATION VARCHAR(64)
)

Done.
Done.


[]

In [21]:
%%sql
INSERT INTO location_dimension (ARREST_LOCATION, INCIDENT_LOCATION)
SELECT DISTINCT ARREST_LOCATION, INCIDENT_LOCATION 
FROM copy_bpd_arrests;
   

32283 rows affected.


[]

In [22]:
%%sql
DROP TABLE IF EXISTS arrests_facts;
CREATE TABLE arrests_facts(
  ARREST INTEGER,
  person_key INTEGER,
  time_key INTEGER,
  location_key INTEGER,
  FOREIGN KEY(person_key) REFERENCES person_dimension(person_key),
  FOREIGN KEY(time_key) REFERENCES time_dimension(time_key),
  FOREIGN KEY(location_key) REFERENCES location_dimension(location_key)
)

Done.
Done.


[]

#### Please note that following query may take a few minutes to execute

In [ ]:
%%sql
INSERT INTO arrests_facts (arrest, person_key, time_key, location_key)
SELECT arrests.arrest, person.person_key, time.time_key, location.location_key
FROM copy_bpd_arrests AS arrests, 
person_dimension AS person,
time_dimension AS time,
location_dimension AS location
WHERE person.age = arrests.age
  AND person.sex = arrests.sex
  AND person.race = arrests.race
  AND time.arrest_date = arrests.arrest_date
  AND time.arrest_time = arrests.arrest_time
  AND location.arrest_location = arrests.arrest_location
  AND location.incident_location = arrests.incident_location;

In [32]:
%%sql
SELECT * FROM arrests_facts
LIMIT 10;

10 rows affected.


arrest,person_key,time_key,location_key
16088671,416,2,12883
16088676,416,2,12883
16088669,416,2,12883
16145813,163,5,5776
13683490,42,9,9639
15006676,368,14,12883
15006674,99,14,12883
15121032,92,16,12883
15038023,101,17,16984
15038017,9,17,16984


## Problem 4 - explore your data in its new schema (30 points)

Strictly using the dimensional model tables you defined and populated for Problem 3, explore your data in its new form.  Use any of the query strategies we've seen in class, including transformations, aggregates, subqueries, rollups, and, of course, joins.  Add plots to highlight particular themes that stand out.  Describe your thinking and observations along the way.

#### How many men and how many women have been arrest according to this data set?

We used this question to compare our results from a previous query to verify that everything in our dimension tables and fact table is joined correctly.  The results match!

In [40]:
%%sql

SELECT 
    P.sex,
    COUNT(*)

FROM arrests_facts A
INNER JOIN person_dimension P ON (A.person_key = P.person_key)

GROUP BY
    P.sex

2 rows affected.


sex,count
M,104478
F,24547


#### Does the volume of men and women arrests seem to vary by race?

It seems that the races U and I have a higher proportion of male arrests in comparison to the other races.

In [44]:
%%sql

SELECT 
    P.race,
    P.sex,
    COUNT(*)

FROM arrests_facts A
INNER JOIN person_dimension P ON (A.person_key = P.person_key)

GROUP BY
    P.race, P.sex

10 rows affected.


race,sex,count
W,F,6634
W,M,13824
A,M,259
A,F,81
I,F,38
I,M,334
B,M,87767
B,F,17465
U,M,2294
U,F,329


#### Which locations have the most arrests?

Interestingly, the highest count is for not reported.  Which brings up the question of how accurate those neighborhood crime maps are!

In [32]:
%%sql
SELECT 
    L.ARREST_LOCATION,
    COUNT(*) AS count

FROM arrests_facts A
INNER JOIN location_dimension L ON (A.location_key = L.location_key)

GROUP BY
    L.ARREST_LOCATION
    
ORDER BY count DESC
LIMIT 10;

10 rows affected.


arrest_location,count
NOT_REPORTED,51324
200 N EUTAW ST,447
1600 W NORTH AVE,343
400 E LEXINGTON ST,297
1500 RUSSELL ST,289
300 N EUTAW ST,235
400 E BALTIMORE ST,226
400 W SARATOGA ST,218
5100 REISTERSTOWN RD,207
1500 W NORTH AVE,197


#### Do certain dates seem to have higher crime than others?

From this data, it is hard to tell if there is a pattern.  A more granular level of grain would likely provide a more specific answer to this question.

In [48]:
%%sql
SELECT 
    T.ARREST_DATE,
    COUNT(*) AS count

FROM arrests_facts A
INNER JOIN time_dimension T ON (A.time_key = T.time_key)

GROUP BY
    T.ARREST_DATE
    
ORDER BY count DESC
LIMIT 25;

25 rows affected.


arrest_date,count
2013-08-15 00:00:00,186
2014-01-09 00:00:00,182
2013-05-29 00:00:00,178
2014-04-23 00:00:00,175
2014-03-06 00:00:00,172
2013-10-01 00:00:00,171
2015-04-27 00:00:00,171
2014-06-05 00:00:00,171
2013-03-13 00:00:00,171
2014-06-11 00:00:00,169


## Bonus (10 points for one of A or B)


### Option A - Automating ETL

Consider the work you did to load your original raw dataset and then transform it into a dimensional model for analysis.  What would it take to automate this process?  How often would you need to update the data?  Could you easily automate any data cleaning steps?  What checks would you need to put in place to ensure quality?  Would humans need to be involved, or could you automate it all?

Discuss.

Consider the work you did to load your original raw dataset and then transform it into a dimensional model for analysis.  

#### What would it take to automate this process?  
We would be curious to know if it is possible to create various types of IDs/keys in the master data table in order to avoid the manual process of linking dimension tables and facts together.  Not sure if that is possible, but that form of autmation would likely have tremendous benefit.

Additionally, removing of missing / null values seems like something that could be relatively easy to automate.  For example, someone could write a set of rules for how to deal with various types of values as they come in, so that values such as null would not be present in the database.

Lastly, it seems very possible that something such as parsing date and time into individual variables for day, month, year, hour, and minute would be something that could be automated.  In order to do this, it seems that either:  1)  that data could be collected at a more granular format, or 2)someone could write a function that would take a data frame as a parameter, parse the dates into individual variables, return an updated data frame with the denormalized variables.


#### How often would you need to update the data?  
It seems like this topic could be debated from a number of viewpoints and likely depends on the nature of the business and how often they are pulling analytical reports.  For example, if a business is typically only reviewing sales on a daily basis, it seems reasonable that the data could be collected throughout the day then put through the ETL process at night.


#### Could you easily automate any data cleaning steps?  
As mentioned above, dealing with missing / null values and cleaning up a variable such as date would be quite helpful.


#### What checks would you need to put in place to ensure quality?  
For the missing / null example, I believe the team that is working on the automation would need to consider the breadth of values that are currently being recorded.  They would need to ensure that there are clear approaches for dealing with all of them.  The team would also likely need to try to plan for the unexpected by trying to identify the types of values that have not been encounted thus far, but may encounter in the future.

#### Would humans need to be involved, or could you automate it all?
It seems that the process could be largely automated, but as mentioned in the previous response the process will still need some level of human interaction.  Going back to the missing / null value example, someone will need to ensure that the set of rules that had been put into place to deal with these values is still working properly and that issues and/or edge cases have arose but are not being dealt with.


### Option B - Augmenting dimensions

Considering your dimensional model, what external data could you find to augment it?  As in the case of bike trips, where weather might provide an interesting added dimension/context not present in the original data, you can probably find another source of data to complement your own model.  Identify one such source and add it to your model, demonstrating its value with a few new queries.